In [2]:
# https://pypi.org/project/mysql-connector-python/
# !pip install mysql-connector-python

In [8]:
import mysql.connector as connection

In [14]:
try:
    mydb = connection.connect(host='localhost',
                             user='root',
                             password='root')
    print(f"Connected to : {mydb}")
    
    cursor = mydb.cursor()
    query = "show databases;"
    cursor.execute(query)
    print(f"{cursor.fetchall()}")
except Exception as e:
    mydb.close()
    print(f"Exception : {e}")
    

Connected to : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000027AB4AAC730>
[('information_schema',), ('mysql',), ('performance_schema',), ('sakila',), ('sys',), ('world',)]


In [21]:
# CREATE DATABASE
query = 'CREATE database project_practice;'
cursor.execute(query)
print(f"CREATE DATABASE : {cursor.fetchall()}")

query = "show databases;"
cursor.execute(query)
print(f"SHOW DATABASE :")
for db_name in cursor.fetchall():
    print(f"{db_name[0]}")

# TO DROP/DELETE DATABASE
# query = 'drop database project_practice;'

CREATE DATABASE : []
SHOW DATABASE :
information_schema
mysql
performance_schema
project_practice
sakila
sys
world


In [27]:
# CREATE TABLE
mydb = connection.connect(host='localhost',
                          user='root',
                          password='root', 
                          database='project_practice')
cursor = mydb.cursor()
print(f"Connected to : {mydb} , {cursor}")

query = """create table if not exists student_details (stud_id int(10) auto_increment primary key, first_name varchar(20),
last_name varchar(20), registration_date date, class varchar(20), section varchar(20));"""
# query = """use project_practice;
# create table student_details (stud_id int(10) auto_increment primary key, first_name varchar(20),
# last_name varchar(20), registration_date date, class varchar(20), section varchar(20));"""
cursor.execute(query)
print(f"SHOW TABLE : {cursor.fetchall()}")

Connected to : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000027AB4A67DF0> , CMySQLCursor: (Nothing executed yet)
SHOW TABLE : []
